# Milano Mobile Traffic Prediction — End-to-End
Bu notebook, Milano CDR verisi ile kısa vadeli mobil internet trafiğini tahmin etmek için uçtan uca veri hazırlama, özellik çıkarımı, modelleme ve değerlendirmeyi içerir.

Adımlar:
- Veri hazırlama (eksik/duplike temizliği, birleştirme)
- Özellik mühendisliği (zaman, lag/rolling, döngüsel kodlama)
- Baseline modeller (Naive = lag_1, Moving Average)
- ML modeller (RandomForest; opsiyonel: XGBoost)
- Zaman-temelli ayrım, metrikler (MAE, RMSE, R²), görseller ve artefaktlar

In [47]:
# Setup & Paths
import os, sys, runpy, json
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import joblib

# Resolve project root (works if run from scripts/ or project root)
ROOT = Path.cwd()
if ROOT.name == 'scripts' and (ROOT.parent / 'data').exists():
    ROOT = ROOT.parent
elif (ROOT / 'scripts').exists() and (ROOT / 'data').exists():
    pass

DATA_RAW = ROOT / 'data' / 'raw'
DATA_PROCESSED = ROOT / 'data' / 'processed'
RESULTS = ROOT / 'results'
SCRIPTS = ROOT / 'scripts'

DATA_PROCESSED.mkdir(parents=True, exist_ok=True)
RESULTS.mkdir(parents=True, exist_ok=True)

PROCESSED_FILE = DATA_PROCESSED / 'milano_internet_combined.csv'
FEATURES_FILE  = DATA_PROCESSED / 'milano_features.csv'

print('ROOT     :', ROOT)
print('RAW      :', DATA_RAW)
print('PROCESSED:', DATA_PROCESSED)
print('RESULTS  :', RESULTS)

ROOT     : /Users/hamza/Documents/MobilCom/mobile_dataset
RAW      : /Users/hamza/Documents/MobilCom/mobile_dataset/data/raw
PROCESSED: /Users/hamza/Documents/MobilCom/mobile_dataset/data/processed
RESULTS  : /Users/hamza/Documents/MobilCom/mobile_dataset/results


In [48]:
# Build processed/feature data if missing
if not PROCESSED_FILE.exists():
    print('Processed CSV not found → running prepare_data.py')
    runpy.run_path(str(SCRIPTS / 'prepare_data.py'), run_name='__main__')
else:
    print('OK: processed CSV exists')

if not FEATURES_FILE.exists():
    print('Features CSV not found → running feature_engineering.py')
    runpy.run_path(str(SCRIPTS / 'feature_engineering.py'), run_name='__main__')
else:
    print('OK: features CSV exists')

OK: processed CSV exists
OK: features CSV exists


In [49]:
# Load features (may take time due to size)
df = pd.read_csv(FEATURES_FILE)
df['time_interval'] = pd.to_datetime(df['time_interval'])
df = df.sort_values(['square_id','time_interval']).reset_index(drop=True)
print('Rows:', len(df), 'Cols:', len(df.columns))
display(df.head())
print('Date range:', df['time_interval'].min(), '→', df['time_interval'].max())
print('Cells:', df['square_id'].nunique())

Rows: 1619994 Cols: 18


,time_interval,square_id,internet_traffic,hour,minute,day_of_week,is_weekend,hour_sin,hour_cos,dow_sin,dow_cos,lag_1,lag_2,lag_6,roll_mean_3,roll_mean_6,roll_mean_12,roll_std_6
0,2013-11-01 06:00:00,1,35.4161,6,0,4,0,1.000000,6.123234e-17,-0.433884,-0.900969,34.8416,31.3769,57.7990,33.080200,40.382267,40.382267,9.832804
1,2013-11-01 07:00:00,1,42.9335,7,0,4,0,0.965926,-2.588190e-01,-0.433884,-0.900969,35.4161,34.8416,44.0469,33.878200,36.651783,39.672814,4.924253
2,2013-11-01 08:00:00,1,59.8808,8,0,4,0,0.866025,-5.000000e-01,-0.433884,-0.900969,42.9335,35.4161,41.2071,37.730400,36.466217,40.080400,4.600151
3,2013-11-01 09:00:00,1,71.1355,9,0,4,0,0.707107,-7.071068e-01,-0.433884,-0.900969,59.8808,42.9335,33.0221,46.076800,39.578500,42.280444,10.709396
4,2013-11-01 10:00:00,1,81.7390,10,0,4,0,0.500000,-8.660254e-01,-0.433884,-0.900969,71.1355,59.8808,31.3769,57.983267,45.930733,45.165950,16.026281


Date range: 2013-11-01 06:00:00 → 2013-11-07 23:00:00
Cells: 10000


In [50]:
# Quick EDA
plt.figure(figsize=(12,4))
mean_ts = df.groupby('time_interval')['internet_traffic'].mean().reset_index()
plt.plot(mean_ts['time_interval'], mean_ts['internet_traffic'], color='tab:orange')
plt.title('Average Internet Traffic Over Time')
plt.xlabel('Time'); plt.ylabel('Avg Traffic'); plt.grid(True); plt.tight_layout()
p1 = RESULTS / 'avg_traffic_over_time.png'
plt.savefig(p1); plt.close(); print('Saved:', p1)

plt.figure(figsize=(6,4))
sns.histplot(df['internet_traffic'], bins=50)
plt.title('Distribution of Internet Traffic')
plt.tight_layout()
p2 = RESULTS / 'traffic_distribution.png'
plt.savefig(p2); plt.close(); print('Saved:', p2)

Saved: /Users/hamza/Documents/MobilCom/mobile_dataset/results/avg_traffic_over_time.png
Saved: /Users/hamza/Documents/MobilCom/mobile_dataset/results/traffic_distribution.png


In [51]:
# Time-based split (last 20% as test)
unique_times = df['time_interval'].sort_values().unique()
cut_idx = int(len(unique_times)*0.8)
cut_time = unique_times[cut_idx]
print('Cut time:', cut_time)

train_df = df[df['time_interval'] < cut_time].copy()
test_df  = df[df['time_interval'] >= cut_time].copy()
print('Train rows:', len(train_df), 'Test rows:', len(test_df))

target = 'internet_traffic'
drop_cols = ['time_interval','square_id', target]
features = [c for c in df.columns if c not in drop_cols]
print('Feature count:', len(features))

Cut time: 2013-11-06 15:00:00
Train rows: 1289995 Test rows: 329999
Feature count: 15


In [52]:
# Fast mode toggle (speeds up training/iteration)
FAST_TRAIN = True           # set False for full training
FAST_SAMPLE_FRAC = 0.25      # use 50% of training rows when FAST_TRAIN
print('Fast mode:', FAST_TRAIN, '| sample frac:', FAST_SAMPLE_FRAC)

Fast mode: True | sample frac: 0.25


In [53]:
# Baseline 1: Naive (lag_1)
if 'lag_1' in features:
    y_pred_b = test_df['lag_1'].to_numpy()
    y_true   = test_df[target].to_numpy()
    mae_b  = mean_absolute_error(y_true, y_pred_b)
    rmse_b = np.sqrt(mean_squared_error(y_true, y_pred_b))
    r2_b   = r2_score(y_true, y_pred_b)
    print(f'Baseline lag_1 → MAE={mae_b:.4f} RMSE={rmse_b:.4f} R²={r2_b:.4f}')
else:
    print('lag_1 not found; skipping baseline.')

Baseline lag_1 → MAE=62.8069 RMSE=171.3403 R²=0.9650


In [54]:
# Baseline 2: Moving Average (last k lags)
def moving_average_pred(g, k=3):
    # Prefer shifted rolling mean columns (non-leaky)
    col = f'roll_mean_{k}'
    if col in g.columns:
        return g[col].to_numpy()
    # Fallback: average of available lag_1..lag_k
    lag_cols = [f'lag_{i}' for i in [1,2,3,4,5,6] if f'lag_{i}' in g.columns and i<=k]
    if lag_cols:
        return g[lag_cols].mean(axis=1).to_numpy()
    # Worst-case fallback
    return np.full(len(g), g['internet_traffic'].mean())

ma_metrics = {}
for k in [3,6,12]:
    y_pred_ma = moving_average_pred(test_df, k=min(k,6))
    y_true    = test_df[target].to_numpy()
    mae = mean_absolute_error(y_true, y_pred_ma)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred_ma))
    r2 = r2_score(y_true, y_pred_ma)
    ma_metrics[k] = { 'mae': float(mae), 'rmse': float(rmse), 'r2': float(r2) }
    print(f'MovingAverage({k}) → MAE={mae:.4f} RMSE={rmse:.4f} R²={r2:.4f}')

MovingAverage(3) → MAE=91.5670 RMSE=261.7854 R²=0.9183
MovingAverage(6) → MAE=129.1227 RMSE=374.5573 R²=0.8327
MovingAverage(12) → MAE=129.1227 RMSE=374.5573 R²=0.8327


In [55]:
# Train RandomForestRegressor
X_train = train_df[features].values
y_train = train_df[target].values
X_test  = test_df[features].values
y_test  = test_df[target].values

# Fast mode controls
try:
    FAST_TRAIN
except NameError:
    FAST_TRAIN = False
FAST_SAMPLE_FRAC = globals().get('FAST_SAMPLE_FRAC', 1.0)

# Optional subsampling for faster iteration
if FAST_TRAIN and 0.0 < FAST_SAMPLE_FRAC < 1.0:
    import numpy as np
    rs = np.random.RandomState(42)
    n = len(X_train)
    k = max(1, int(n * FAST_SAMPLE_FRAC))
    idx = rs.choice(n, size=k, replace=False)
    X_train = X_train[idx]
    y_train = y_train[idx]
    print(f"[FAST] Subsampled train: {k}/{n} rows")

rf = RandomForestRegressor(
    n_estimators=100 if FAST_TRAIN else 200,
    max_depth=12 if FAST_TRAIN else None,
    min_samples_leaf=5 if FAST_TRAIN else 1,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f'RandomForest → MAE={mae:.4f} RMSE={rmse:.4f} R²={r2:.4f}')

[FAST] Subsampled train: 322498/1289995 rows
RandomForest → MAE=50.4265 RMSE=134.2992 R²=0.9785


In [56]:
# Optional: XGBoost (graceful if not installed)
try:
    import xgboost as xgb
    # Fast mode controls
    try:
        FAST_TRAIN
    except NameError:
        FAST_TRAIN = False
    n_estimators = 300 if FAST_TRAIN else 500
    xgbr = xgb.XGBRegressor(
        n_estimators=n_estimators,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    )
    xgbr.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    yx = xgbr.predict(X_test)
    mae_x = mean_absolute_error(y_test, yx)
    rmse_x = np.sqrt(mean_squared_error(y_test, yx))
    r2_x = r2_score(y_test, yx)
    print(f'XGBoost → MAE={mae_x:.4f} RMSE={rmse_x:.4f} R²={r2_x:.4f} | n_estimators={n_estimators}')
except Exception as e:
    print('XGBoost not available or failed to train:', e)

XGBoost → MAE=57.9307 RMSE=216.8824 R²=0.9439 | n_estimators=300


In [57]:
# Evaluation: per-cell metrics and plots
test_eval = test_df[['square_id','time_interval', target]].copy()
test_eval['pred_rf'] = y_pred

def rmse_f(a,b): return float(np.sqrt(np.mean((a-b)**2)))
per_cell = (test_eval.groupby('square_id')
           .apply(lambda g: pd.Series({
               'MAE': mean_absolute_error(g[target], g['pred_rf']),
               'RMSE': rmse_f(g[target].to_numpy(), g['pred_rf'].to_numpy()),
               'R2': r2_score(g[target], g['pred_rf'])
           }))
          .reset_index())
display(per_cell.sort_values('RMSE').head())
out_csv = RESULTS / 'per_cell_metrics.csv'
per_cell.to_csv(out_csv, index=False)
print('Saved:', out_csv)

# Example predicted vs actual plot for a sample cell (correct alignment by cell & time)
sample_cell = int(per_cell.sort_values('RMSE').iloc[0]['square_id']) if len(per_cell) else int(test_df['square_id'].iloc[0])
# Select predictions already aligned per row, then filter by the chosen cell
g = test_eval[test_eval['square_id'] == sample_cell].copy()
g = g.sort_values('time_interval')
plt.figure(figsize=(12,4))
plt.plot(g['time_interval'], g[target], label='Actual')
plt.plot(g['time_interval'], g['pred_rf'], label='RF Pred', alpha=0.8)
plt.title(f'Cell {int(sample_cell)} — Actual vs Predicted')
plt.legend(); plt.tight_layout()
pv = RESULTS / f'pred_vs_actual_cell_{int(sample_cell)}2.png'
plt.savefig(pv); plt.close(); print('Saved:', pv)

/var/folders/f_/xz3vy0ns22jdm26zz2tpj82w0000gn/T/ipykernel_13868/2266332149.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,square_id,MAE,RMSE,R2
111,112,0.429318,0.511431,0.305492
2800,2801,0.443042,0.535801,-0.463523
5238,5239,0.630211,0.652103,-2.866016
1206,1207,0.814531,0.975162,0.220599
5338,5339,0.780687,1.455482,-1.041601


Saved: /Users/hamza/Documents/MobilCom/mobile_dataset/results/per_cell_metrics.csv
Saved: /Users/hamza/Documents/MobilCom/mobile_dataset/results/pred_vs_actual_cell_1122.png


In [58]:
# Save model and summary metrics
metrics = {}
# RF
try:
    metrics['rf_mae']  = float(mae)
    metrics['rf_rmse'] = float(rmse)
    metrics['rf_r2']   = float(r2)
except NameError:
    pass
# XGB
try:
    metrics['xgb_mae']  = float(mae_x)
    metrics['xgb_rmse'] = float(rmse_x)
    metrics['xgb_r2']   = float(r2_x)
except Exception:
    pass
# Baseline - Naive
try:
    metrics['baseline_naive'] = {
        'mae': float(mae_b), 'rmse': float(rmse_b), 'r2': float(r2_b)
    }
except Exception:
    pass
# Baseline - Moving Average (3/6/12)
try:
    metrics['baseline_ma'] = {
        'k3': ma_metrics.get(3, {}),
        'k6': ma_metrics.get(6, {}),
        'k12': ma_metrics.get(12, {})
    }
except Exception:
    pass

MODEL_PATH = RESULTS / 'rf_model.pkl'
try:
    joblib.dump(rf, MODEL_PATH)
    print('Saved model:', MODEL_PATH)
except Exception as e:
    print('Model save skipped:', e)

with open(RESULTS / 'metrics.json', 'w', encoding='utf-8') as f:
    json.dump(metrics, f, indent=2)
print('Saved metrics:', RESULTS / 'metrics.json')

Saved model: /Users/hamza/Documents/MobilCom/mobile_dataset/results/rf_model.pkl
Saved metrics: /Users/hamza/Documents/MobilCom/mobile_dataset/results/metrics.json


## Next steps
- Zaman bazlı çapraz doğrulama (expanding window) ekleyin.
- XGBoost’u kalibre edip RF ile kıyas tablosu üretin.
- Prophet/ARIMA’yı en iyi kapsamalı 5–10 hücrede deneyin.
- Hata analizi grafiklerini (residual dağılımı, zaman/hucre ısı haritaları) ekleyin.
- Sonuçları raporlayıp sunum slaytlarını hazırlayın.

In [59]:
# Time-based cross-validation (expanding windows) for RF/XGB
import numpy as np, pandas as pd, json
from pathlib import Path

def make_folds_times(times: pd.Series, cut_points=(0.6,0.7,0.8,0.9)):
    t = np.sort(times.unique())
    n = len(t)
    idx = [int(n*c) for c in cut_points]
    folds = []
    for i in range(len(idx)-1):
        tr_end = t[idx[i]]
        val_end = t[idx[i+1]]
        folds.append((tr_end, val_end))
    return folds

def build_xy(df_part, feature_cols, target_col):
    X = df_part[feature_cols].values
    y = df_part[target_col].values
    return X, y

# Build folds on the training period only
folds = make_folds_times(train_df['time_interval'])
print('CV folds:', [(str(a), str(b)) for a,b in folds])

cv_results = {'rf': [], 'xgb': []}
for fold_i, (tr_end, val_end) in enumerate(folds, 1):
    tr = train_df[train_df['time_interval'] <= tr_end]
    va = train_df[(train_df['time_interval'] > tr_end) & (train_df['time_interval'] <= val_end)]
    X_tr, y_tr = build_xy(tr, features, target)
    X_va, y_va = build_xy(va, features, target)
    # Optional subsample in Fast Mode
    try:
        use_fast = bool(FAST_TRAIN)
        frac = float(FAST_SAMPLE_FRAC) if 'FAST_SAMPLE_FRAC' in globals() else 1.0
    except Exception:
        use_fast = False; frac = 1.0
    if use_fast and 0.0 < frac < 1.0 and len(X_tr) > 1:
        rs = np.random.RandomState(42)
        k = max(1, int(len(X_tr)*frac))
        idx = rs.choice(len(X_tr), size=k, replace=False)
        X_tr, y_tr = X_tr[idx], y_tr[idx]
    # RandomForest
    rf_cv = RandomForestRegressor(n_estimators=100 if use_fast else 200, max_depth=12 if use_fast else None, min_samples_leaf=5 if use_fast else 1, random_state=42, n_jobs=-1)
    rf_cv.fit(X_tr, y_tr)
    pr = rf_cv.predict(X_va)
    mae_rf = mean_absolute_error(y_va, pr)
    rmse_rf = np.sqrt(mean_squared_error(y_va, pr))
    r2_rf = r2_score(y_va, pr)
    cv_results['rf'].append({'fold': fold_i, 'mae': float(mae_rf), 'rmse': float(rmse_rf), 'r2': float(r2_rf)})
    # XGBoost (if available)
    try:
        import xgboost as xgb
        xgb_cv = xgb.XGBRegressor(n_estimators=300 if use_fast else 500, max_depth=8, learning_rate=0.05, subsample=0.8, colsample_bytree=0.8, random_state=42, n_jobs=-1)
        xgb_cv.fit(X_tr, y_tr, eval_set=[(X_va, y_va)], verbose=False)
        px = xgb_cv.predict(X_va)
        mae_x = mean_absolute_error(y_va, px)
        rmse_x = np.sqrt(mean_squared_error(y_va, px))
        r2_x = r2_score(y_va, px)
        cv_results['xgb'].append({'fold': fold_i, 'mae': float(mae_x), 'rmse': float(rmse_x), 'r2': float(r2_x)})
    except Exception as e:
        # XGB not installed or failed; skip
        pass

# Summaries
def summarize(name, rows):
    import numpy as np
    if not rows:
        return None
    mae = np.array([r['mae'] for r in rows])
    rmse = np.array([r['rmse'] for r in rows])
    r2 = np.array([r['r2'] for r in rows])
    return {
        'cv_mae_mean': float(mae.mean()), 'cv_mae_std': float(mae.std()),
        'cv_rmse_mean': float(rmse.mean()), 'cv_rmse_std': float(rmse.std()),
        'cv_r2_mean': float(r2.mean()), 'cv_r2_std': float(r2.std()),
        'folds': rows
    }

cv_summary = {
    'rf': summarize('rf', cv_results['rf']),
    'xgb': summarize('xgb', cv_results['xgb'])
}
print('CV summary:', json.dumps(cv_summary, indent=2))

# Save CSV comparison
rows = []
def row_from(name, summ):
    if not summ:
        return {'model': name}
    return {'model': name, 'cv_rmse_mean': summ['cv_rmse_mean'], 'cv_rmse_std': summ['cv_rmse_std'], 'cv_mae_mean': summ['cv_mae_mean'], 'cv_mae_std': summ['cv_mae_std']}
rows.append(row_from('rf', cv_summary['rf']))
rows.append(row_from('xgb', cv_summary['xgb']))
cmp_df = pd.DataFrame(rows)
cmp_path = RESULTS / 'model_comparison.csv'
cmp_df.to_csv(cmp_path, index=False)
print('Saved:', cmp_path)

CV folds: [('2013-11-04T11:00:00.000000000', '2013-11-05T00:00:00.000000000'), ('2013-11-05T00:00:00.000000000', '2013-11-05T13:00:00.000000000'), ('2013-11-05T13:00:00.000000000', '2013-11-06T02:00:00.000000000')]
CV summary: {
  "rf": {
    "cv_mae_mean": 52.72824542732252,
    "cv_mae_std": 4.222033409879141,
    "cv_rmse_mean": 144.0123321380179,
    "cv_rmse_std": 10.895380419782079,
    "cv_r2_mean": 0.9743349661421989,
    "cv_r2_std": 0.0037946212608808184,
    "folds": [
      {
        "fold": 1,
        "mae": 58.28449840348217,
        "rmse": 157.24458783137925,
        "r2": 0.9737936827035827
      },
      {
        "fold": 2,
        "mae": 48.0568707199404,
        "rmse": 144.23320134317117,
        "r2": 0.9699818664165729
      },
      {
        "fold": 3,
        "mae": 51.84336715854499,
        "rmse": 130.5592072395033,
        "r2": 0.9792293493064415
      }
    ]
  },
  "xgb": {
    "cv_mae_mean": 63.55168902759841,
    "cv_mae_std": 7.110552037458202,
    

In [60]:
# Select best model based on CV RMSE (lower is better) and update metrics.json
import json, os
metrics_path = RESULTS / 'metrics.json'
try:
    with open(metrics_path, 'r', encoding='utf-8') as f:
        existing = json.load(f)
except Exception:
    existing = {}

best_name = None
best_rmse = float('inf')
for name in ['rf','xgb']:
    summ = cv_summary.get(name)
    if summ and summ['cv_rmse_mean'] < best_rmse:
        best_rmse = summ['cv_rmse_mean']
        best_name = name

existing['cv'] = cv_summary
existing['model_comparison'] = {'selected': best_name, 'selected_cv_rmse_mean': best_rmse}
print('Selected model:', best_name, 'CV RMSE:', best_rmse)

# Save best model artifact if available
try:
    if best_name == 'rf':
        joblib.dump(rf, RESULTS / 'best_model.pkl')
    elif best_name == 'xgb':
        joblib.dump(xgbr, RESULTS / 'best_model.pkl')
    print('Saved best model as best_model.pkl')
except Exception as e:
    print('Best model save skipped:', e)

with open(metrics_path, 'w', encoding='utf-8') as f:
    json.dump(existing, f, indent=2)
print('Updated metrics.json with CV summaries and selection.')

Selected model: rf CV RMSE: 144.0123321380179
Saved best model as best_model.pkl
Updated metrics.json with CV summaries and selection.
